## Disclaimer

**Environment:** This notebook has been developped for running with:

- Julia v1.11.x
- JuMP.jl v1.23.x
- GLPK.jl v1.2.1
- HiGHS v1.12.1

**Author:** Xavier Gandibleux, Nantes (France)


----

## Exercise 2 
For 
- the unidimensional 01 knapsack problem, and
- the corresponding numerical instance
  
from Exercise 1, 
1. Write a function in Julia which computes the linear relaxation of the problem using the Dantzig bound.
2. Compute the linear relaxation for this instance.

<ins>Reminder:</ins> 

Compute:
$$ 
\begin{aligned}
\max\quad & \sum_{j=1}^{n} p_{j} x_{j}\\
\text{Subject to} \quad & \sum_{j=1}^{n} w_{j} x_{j} \leq c\\
 & 0 \le x_j \le 1 \qquad  j=1,\dots ,n\\
\end{aligned} 
$$
I.e., with the following non-restrictive hypotheses:
- $p_j$, $w_j$ et $C$ : positive integers
- $\displaystyle{\sum_{j=1}^{n} w_j> C}$
-  $w_j \le C, \ j=1, \dots , n$
- $ \frac{p_1}{w_1} \ge  \frac{p_2}{w_2} \ge  \frac{p_3}{w_3} \ge ...  \ge \frac{p_n}{w_n} $

compute:
- $s = \min \Big\{ j  \ \mid \ \displaystyle{\sum_{i=1}^{j} w_i > C} \Big\}$
- $
\left\{\begin{array}{ccc}
 \bar{x}_j = 1 \hspace{7mm} \mbox{for } j=1, ... , \ s-1   \\
 \bar{x}_j = 0 \hspace{7mm} \mbox{for } j=s+1, \dots ,n  \\
 \ \ \ \ \bar{x}_s = \frac{\bar{C}}{w_s} \hspace{5mm} \mbox{avec } \bar{C} = C - \displaystyle{\sum_{j=1}^{s-1} w_j}\\
\end{array}\right.
$


## Answers:

----
### 1. Function which computes the linear relaxation of the problem using the Dantzig bound.

In [1]:
using Printf

function computeLP_U01KP( p::Vector{Int64}, w::Vector{Int64}, c::Int64 )

    n = length( p )
    xLP = zeros( Float64, n )
    
    # compute the utilities and reorder the items accordingly
    u = p ./ w
    reord = sortperm( u, rev=true )

    # identify the last item integrally selected
    zLP = 0 ; bar_c = c; s = 1 
    while ( s <= n ) && ( bar_c - w[reord[s]]  >= 0 )
         xLP[reord[s]] = 1.0
         zLP = zLP + p[reord[s]]   
         bar_c = bar_c - w[reord[s]] 
         s = s + 1
    end
    s = s - 1
                
    # constraint not saturated => add the fractionnal part of the blocking item valuated
    if ( bar_c > 0 ) && ( s < n )
        xLP[reord[s+1]] = bar_c / w[reord[s+1]]        
        zLP = zLP + xLP[reord[s+1]] * p[reord[s+1]]  
    end
    
    for i = 1:n
        @printf(" %d (%d %d %.2f) %4.2f \n", reord[i], p[reord[i]], w[reord[i]], u[reord[i]], xLP[reord[i]])
    end
       
    return zLP, xLP
end

computeLP_U01KP (generic function with 1 method)

----
### 2. Compute the linear relaxation for an instance.

In [2]:
p = [ 5, 3, 2, 7, 4 ]
w = [ 2, 8, 4, 2, 5 ]
c = 10

computeLP_U01KP( p, w, c )

 4 (7 2 3.50) 1.00 
 1 (5 2 2.50) 1.00 
 5 (4 5 0.80) 1.00 
 3 (2 4 0.50) 0.25 
 2 (3 8 0.38) 0.00 


(16.5, [1.0, 0.0, 0.25, 1.0, 1.0])